# SVMによるボクセルごとの学習と性能評価（時系列解析）
----

引数：raw_al45.csv/raw_al135.csvがあるディレクトリまでのパス

----

入力：raw_rest.csv/raw_tapping.csv

----

出力：ACCURACY[loo or k_cv]_VOXtimeseries[1時系列あたりのスキャン数]_SVM.csv ボクセルごとの識別性能評価結果一覧

----
ボクセルごとにあるデータにおけるNスキャン分の時系列データをテストデータ，テストデータを除いた残りのデータにおけるNスキャン分の時系列データ1ずつずらしながらを学習データとして取得し，SVMを用いて学習，交差検証法を用いて識別性能評価を行う．  
ベクトル：各ボクセルにおけるデータにおけるNスキャン分の時系列データ

In [1]:
print('############ ML_SVM_VOXtimeseries_kCV.py program excution ############')

############ ML_SVM_VOXtimeseries_kCV.py program excution ############


In [2]:
import numpy as np
import pandas as pd
import sys

from sklearn import svm

In [2]:
#args = sys.argv
#PATH = args[1]

# jupyter notebookのときはここで指定
PATH = '../State-2fe_MaskBrodmann/20181029rn/mb/RawData/'

# 検証手法
col_name = 'leave-one-out'

kCV = 10


In [3]:
headcoil = PATH.split('/')[3]


if headcoil == 'mb':
    
    # 1時系列あたりスキャン数
    N = 30
    
else :

    # 1時系列あたりスキャン数
    N = 10

print("TimeSeries(Scan Num) : " + str(N))

TimeSeries(Scan Num) : 30


## TsShift関数
引数としてあるボクセルにおける全試行分のデータをdata，タスクを見分けるための番号をlabelに受け取る．  
各試行で1ずつずらしながらNスキャン分の時系列データを取得する．全試行の時系列データをまとめて返す．

In [5]:
def TsShift(data, label):
    
    # 1ボクセルあたりで取得する時系列データ格納用データフレーム
    ts_all = pd.DataFrame(index = [], columns = [])

    # 時系列として扱う区間の始まり
    ts_fst = 0

    # 時系列として扱う区間の終わり
    ts_end = N

    # 1ずつずらしながら時系列データを取得，結合
    while ts_end <= len(data):

        # 時系列として扱う区間の始まり，区間の終わり，その区間（Nスキャン分）の時系列データの順でリスト化
        ts = [label] + [ts_fst + 1] + [ts_end] + list(data[ts_fst:ts_end])

        # データフレーム化，結合
        ts = pd.DataFrame(ts).T

        ts_all = pd.concat([ts_all, ts])

        # 1ずつずらす
        ts_fst = ts_fst + 1
        ts_end = ts_end + 1
            
            
    return ts_all

## SVM_kCV関数
学習と評価に用いるデータをdataで受け取り．  
データからテストデータ，テストデータラベル，教師データ，教師データラベルを生成．  
テストデータはdataにおける全時系列データ，教師データは条件に当てはまる時系列データであり，  
10分割交差検証法を用いて識別率を算出，平均を計算し，main関数へ返す．

In [6]:
def SVM_kCV(data):

    scores = np.zeros(kCV)

    testNum = len(data)//kCV
    
    print('allvec = ' + str(len(data)))

    for i in range(kCV):
        
        # kCV回目の評価では端数分のデータを入れる
        if i == (kCV-1):
            
            print("--------" + str(i + 1) + " / " + str(kCV) + "---------")
            
            # テストデータの情報（タスク，試行数，時系列データの始まり，終わり）を取得
            test_labels = data.iloc[(i * testNum):(len(data)), 0]
            test_Flabel = data.iloc[i * testNum, 0]
            test_Elabel = data.iloc[(len(data) - 1), 0]

            test_fst = data.iloc[i * testNum, 2]
            test_end = data.iloc[(len(data) - 1), 3]
            
            ###### テストデータ

            # 時系列データのみを抽出
            X_test = data.iloc[(i * testNum):(len(data)), 3:len(data.columns)]

            
            
        else:
        
            print("--------" + str(i + 1) + " / " + str(kCV) + "---------")

            # テストデータの情報（タスク，試行数，時系列データの始まり，終わり）を取得
            test_labels = data.iloc[(i * testNum):((i + 1) * testNum), 0]
            test_Flabel = data.iloc[i * testNum, 0]
            test_Elabel = data.iloc[((i + 1) * testNum), 0]

            test_fst = data.iloc[i * testNum, 2]
            test_end = data.iloc[(((i + 1) * testNum) - 1), 3]


            ###### テストデータ

            # 時系列データのみを抽出
            X_test = data.iloc[(i * testNum):((i + 1) * testNum), 3:len(data.columns)]
        
        print('testNum : ' + str(len(X_test)))
            

        # ベクトル化
        X_test = X_test.as_matrix()
        # ラベルを作成
        y_test = np.array(list(test_labels))


        ###### 教師データ

        # テストデータではない，テストデータに含まれるZ-scoreを含まないものを取得

        if test_Flabel == test_Elabel:

            test_label = test_Flabel
            
            traindata = data[(data['label'] != test_label) | (data['fst'] > test_end) | (data['end'] < test_fst)]

        else:
            
            traindata = data[((data['label'] == test_Flabel) & (data['end'] < test_fst)) | ((data['label'] == test_Elabel) & (data['fst'] > test_end))]

 
        print('trainNum : ' str(len(traindata)))
    
        # 時系列データのみを抽出
        X_train = traindata.iloc[:, 3:len(data.columns)]

        # ベクトル化
        X_train = X_train.as_matrix()

        # ラベルを作成
        y_train = np.array(list(traindata['label']))

        # 線形SVMのインスタンスを生成
        model = svm.SVC(kernel = 'linear', C=1)

        # モデルの学習
        model.fit(X_train, y_train)

        # 評価結果を格納
        scores[i] = model.score(X_test, y_test)
        
    # 評価結果の平均を求める
    result = scores.mean()
              
    
    # パーセント表記へ
    result = round(result * 100, 1)
    
    print(str(scores) + '\n')
    
    return result


In [ ]:
if __name__ == '__main__':
    
    # 読み込みたいファイルのパス
    PATH_rest = PATH + 'raw_rest.csv'
    PATH_tapping = PATH + 'raw_tapping.csv'
    
    # csvファイル読み込み
    # headerは設定せず，転置後にset_index()する（header = 0にすると列名が変えられる）
    rest = pd.read_csv(PATH_rest, header = None, index_col = None).T
    rest.columns = range(0, len(rest.columns))
    rest = rest.set_index(0)
    
    tapping = pd.read_csv(PATH_tapping, header = None, index_col = None).T
    tapping.columns = range(0, len(tapping.columns))
    tapping = tapping.set_index(0)

In [365]:
    # ボクセル数
    voxNum = len(rest)
    
    # 全ボクセルの識別率を格納するデータフレーム
    voxAc = pd.DataFrame(index = range(voxNum), columns = [col_name])
    
    counter = 0
    csvcounter = 0
    voxNames = []
    
    
    for voxNo in range(voxNum):
    
    
        voxName = 'Voxel' + str(voxNo + 1)


        print(voxName + '( ' + str(counter) + ' / ' + str(voxNum) + ' )')

        # ボクセルのデータを取得
        restVox = rest.loc[voxName]
        tappingVox = tapping.loc[voxName]
        
        # ボクセルにおける時系列データを取得
        restVoxTs = TsShift(restVox, 0)
        tappingVoxTs = TsShift(tappingVox, 1)
        
        # 全タスクを縦結合
        VoxTs = pd.concat([restVoxTs, tappingVoxTs])
        
        # 0-3列目は条件判定用の要素，要素名をつけておく
        col_names = list(VoxTs.columns)
        col_names[0:3] = ['label', 'fst', 'end']
        VoxTs.columns = col_names
        
        VoxTs.index = range(0,len(VoxTs))
        
        # 学習と評価
        result_vox = SVM_kCV(VoxTs)
        
        print(result_vox)
        
        # データフレームに格納
        voxAc.at[voxNo, :] = result_vox
        
        
        # 途中経過見る用
        # 何ボクセルで一度出力するか
        midNum = 1000

        if (counter % midNum == 0) and (counter != 0):

            PATH_test = PATH + 'ACMID' + str(csvcounter) + '[' + str(kCV) + 'cv]_VOXtimeseries' + str(N) +'_SVM.csv'
            print(PATH_test)
            MidVoxAc = voxAc.iloc[(csvcounter * midNum):((csvcounter + 1) * midNum), :]
            MidVoxAc.index = voxNames[(csvcounter * midNum):((csvcounter + 1) * midNum)]
            MidVoxAc.to_csv(PATH_test, index = True)

            csvcounter = csvcounter + 1
        
        counter = counter + 1
        voxNames = voxNames + [voxName]
        
        
        

Voxel1( 0 / 68255 )
--------1 / 10---------
same label / same run
--------2 / 10---------
same label / not same run
--------3 / 10---------
same label / not same run
--------4 / 10---------
same label / not same run
--------5 / 10---------
same label / same run
--------6 / 10---------
not same label / not same run
--------7 / 10---------
same label / not same run
--------8 / 10---------
same label / not same run
--------9 / 10---------
same label / not same run
--------10 / 10---------
same label / same run
[ 0.5106383   0.21276596  0.42553191  0.          0.40425532  0.23404255
  0.34042553  0.          0.          0.08510638]

22.1
Voxel2( 1 / 68255 )
--------1 / 10---------
same label / same run
--------2 / 10---------
same label / not same run
--------3 / 10---------
same label / not same run
--------4 / 10---------
same label / not same run
--------5 / 10---------
same label / same run
--------6 / 10---------
not same label / not same run
--------7 / 10---------
same label / not s

--------8 / 10---------
same label / not same run
--------9 / 10---------
same label / not same run
--------10 / 10---------
same label / same run
[ 0.59574468  0.19148936  0.42553191  0.95744681  0.57446809  1.
  0.93617021  0.31914894  0.27659574  0.21276596]

54.9
Voxel14( 13 / 68255 )
--------1 / 10---------
same label / same run
--------2 / 10---------
same label / not same run
--------3 / 10---------
same label / not same run
--------4 / 10---------
same label / not same run
--------5 / 10---------
same label / same run
--------6 / 10---------
not same label / not same run
--------7 / 10---------
same label / not same run
--------8 / 10---------
same label / not same run
--------9 / 10---------
same label / not same run
--------10 / 10---------
same label / same run
[ 0.63829787  0.23404255  0.40425532  0.95744681  0.68085106  0.9787234
  0.85106383  0.36170213  0.34042553  0.46808511]

59.1
Voxel15( 14 / 68255 )
--------1 / 10---------
same label / same run
--------2 / 10-------

--------7 / 10---------
same label / not same run
--------8 / 10---------
same label / not same run
--------9 / 10---------
same label / not same run
--------10 / 10---------
same label / same run
[ 0.          0.12765957  0.04255319  0.          0.0212766   0.27659574
  0.          0.          0.          0.14893617]

6.2
../MaskBrodmann2/20181119tm/RawData/ACMID4[10cv]_VOXtimeseries30_SVM.csv
Voxel27( 26 / 68255 )
--------1 / 10---------
same label / same run
--------2 / 10---------
same label / not same run
--------3 / 10---------
same label / not same run
--------4 / 10---------
same label / not same run
--------5 / 10---------
same label / same run
--------6 / 10---------
not same label / not same run
--------7 / 10---------
same label / not same run
--------8 / 10---------
same label / not same run
--------9 / 10---------
same label / not same run
--------10 / 10---------
same label / same run
[ 0.40425532  0.17021277  0.08510638  0.          0.          0.29787234
  0.0212766   

--------1 / 10---------
same label / same run
--------2 / 10---------
same label / not same run
--------3 / 10---------
same label / not same run
--------4 / 10---------
same label / not same run
--------5 / 10---------
same label / same run
--------6 / 10---------
not same label / not same run
--------7 / 10---------
same label / not same run
--------8 / 10---------
same label / not same run
--------9 / 10---------
same label / not same run
--------10 / 10---------
same label / same run
[ 0.53191489  0.10638298  0.36170213  0.          0.34042553  0.27659574
  0.57446809  0.          0.53191489  0.        ]

27.2
Voxel40( 39 / 68255 )
--------1 / 10---------
same label / same run
--------2 / 10---------
same label / not same run
--------3 / 10---------
same label / not same run
--------4 / 10---------
same label / not same run
--------5 / 10---------
same label / same run
--------6 / 10---------
not same label / not same run
--------7 / 10---------
same label / not same run
--------8 

--------8 / 10---------
same label / not same run
--------9 / 10---------
same label / not same run
--------10 / 10---------
same label / same run
[ 0.42553191  0.17021277  0.46808511  0.          0.40425532  0.53191489
  0.82978723  0.          0.57446809  0.        ]

34.0
../MaskBrodmann2/20181119tm/RawData/ACMID9[10cv]_VOXtimeseries30_SVM.csv
Voxel52( 51 / 68255 )
--------1 / 10---------
same label / same run
--------2 / 10---------
same label / not same run
--------3 / 10---------
same label / not same run
--------4 / 10---------
same label / not same run
--------5 / 10---------
same label / same run
--------6 / 10---------
not same label / not same run
--------7 / 10---------
same label / not same run
--------8 / 10---------
same label / not same run
--------9 / 10---------
same label / not same run
--------10 / 10---------
same label / same run
[ 0.40425532  0.08510638  0.19148936  0.          0.46808511  0.55319149
  0.          0.          0.53191489  0.        ]

22.3
Voxel53

...

--------9 / 10---------
same label / not same run
--------10 / 10---------
same label / same run
[ 0.23404255  0.          0.          0.          0.          0.34042553
  0.          0.          0.31914894  0.0212766 ]

9.1
Voxel2726( 2725 / 68255 )
--------1 / 10---------
same label / same run
--------2 / 10---------
same label / not same run
--------3 / 10---------
same label / not same run
--------4 / 10---------
same label / not same run
--------5 / 10---------
same label / same run
--------6 / 10---------
not same label / not same run
--------7 / 10---------
same label / not same run
--------8 / 10---------
same label / not same run
--------9 / 10---------
same label / not same run
--------10 / 10---------
same label / same run
[ 0.38297872  0.          0.          0.          0.          0.29787234
  0.          0.          0.          0.        ]

6.8
../MaskBrodmann2/20181119tm/RawData/ACMID544[10cv]_VOXtimeseries30_SVM.csv
Voxel2727( 2726 / 68255 )
--------1 / 10---------
sam

--------1 / 10---------
same label / same run
--------2 / 10---------
same label / not same run
--------3 / 10---------
same label / not same run
--------4 / 10---------
same label / not same run
--------5 / 10---------
same label / same run
--------6 / 10---------
not same label / not same run
--------7 / 10---------
same label / not same run
--------8 / 10---------
same label / not same run
--------9 / 10---------
same label / not same run
--------10 / 10---------
same label / same run
[ 0.          0.          0.          0.          0.          0.10638298
  0.          0.          0.14893617  0.21276596]

4.7
Voxel2739( 2738 / 68255 )
--------1 / 10---------
same label / same run
--------2 / 10---------
same label / not same run
--------3 / 10---------
same label / not same run
--------4 / 10---------
same label / not same run
--------5 / 10---------
same label / same run
--------6 / 10---------
not same label / not same run
--------7 / 10---------
same label / not same run
-------

--------10 / 10---------
same label / same run
[ 0.04255319  0.          0.          0.          0.          0.57446809
  0.          0.          0.12765957  0.        ]

7.4
Voxel2751( 2750 / 68255 )
--------1 / 10---------
same label / same run
--------2 / 10---------
same label / not same run
--------3 / 10---------
same label / not same run
--------4 / 10---------
same label / not same run
--------5 / 10---------
same label / same run
--------6 / 10---------
not same label / not same run
--------7 / 10---------
same label / not same run
--------8 / 10---------
same label / not same run
--------9 / 10---------
same label / not same run
--------10 / 10---------
same label / same run
[ 0.          0.          0.          0.          0.          0.44680851
  0.          0.          0.          0.        ]

4.5
../MaskBrodmann2/20181119tm/RawData/ACMID549[10cv]_VOXtimeseries30_SVM.csv
Voxel2752( 2751 / 68255 )
--------1 / 10---------
same label / same run
--------2 / 10---------
same la

--------10 / 10---------
same label / same run
[ 0.          0.          0.          0.          0.          0.44680851
  0.          0.          0.23404255  0.0212766 ]

7.0
Voxel2764( 2763 / 68255 )
--------1 / 10---------
same label / same run
--------2 / 10---------
same label / not same run
--------3 / 10---------
same label / not same run
--------4 / 10---------
same label / not same run
--------5 / 10---------
same label / same run
--------6 / 10---------
not same label / not same run
--------7 / 10---------
same label / not same run
--------8 / 10---------
same label / not same run
--------9 / 10---------
same label / not same run


KeyboardInterrupt: 

In [352]:
    # csv書き出し
    PATH_RESULT = PATH + 'ACCURACY[' + str(kCV) + 'CV]_VOXtimeseries' + str(N) +'_SVM.csv'
    voxAc.to_csv(PATH_RESULT, index = True)
    
    # 行名つける
    voxAc.index = voxNames

    # csv書き出し
    PATH_RESULT = PATH + 'ACCURACY[' + str(kCV) + 'CV]_VOXtimeseries' + str(N) +'_SVM.csv'
    voxAc.to_csv(PATH_RESULT, index = True)
    